In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')
tokenizer.convert_tokens_to_ids('Yes')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


8240

In [2]:
FLAGS.folds = 5
df = get_df()
df

,id,anchor,target,context,score,sector,targets,targets_list,context_text,text,num_words,cat,relevance,group,fold,worker
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,act of abating; active catalyst; eliminating p...,"[act of abating, active catalyst, eliminating ...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]abatement of pollution[SEP]HUMAN...,64,2,0.666667,abatement,0,40
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,abatement of pollution; active catalyst; elimi...,"[abatement of pollution, active catalyst, elim...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]act of abating[SEP]HUMAN NECESSI...,64,3,0.888889,abatement,0,25
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,abatement of pollution; act of abating; elimin...,"[abatement of pollution, act of abating, elimi...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]active catalyst[SEP]HUMAN NECESS...,64,1,0.333333,abatement,0,70
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,abatement of pollution; act of abating; active...,"[abatement of pollution, act of abating, activ...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]eliminating process[SEP]HUMAN NE...,64,2,0.666667,abatement,0,20
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,abatement of pollution; act of abating; active...,"[abatement of pollution, act of abating, activ...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]forest region[SEP]HUMAN NECESSIT...,64,0,0.000000,abatement,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden article[SEP]PERFORMING...,61,4,1.000000,wood article,0,65
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden box[SEP]PERFORMING OPE...,61,2,0.666667,wood article,0,10
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden handle[SEP]PERFORMING ...,61,2,0.666667,wood article,0,35
36471,756ec035e694722b,wood article,wooden material,B44,0.75,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden material[SEP]PERFORMIN...,61,3,0.888889,wood article,0,60


In [3]:
df.text.values[0]

'abatement[SEP]abatement of pollution[SEP]HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; COFFEE MILLS; SPICE MILLS; SUCTION CLEANERS IN GENERAL[SEP]act of abating; active catalyst; eliminating process; forest region; greenhouse gases; increased rate; measurement level; minimising sounds; mixing core materials; multi pollution abatement device; noise reduction; pollution abatement; pollution abatement incinerator; pollution certificate; rent abatement; sorbent material; source items pollution abatement technology; stone abutments; tax abatement; water bodies'

In [4]:
xs = []
for i in range(5):
  root = f'../working/offline/10/{i}/deberta-v3'
  x = gezi.load(f'{root}/valid.pkl')
  x['fold'] = np.asarray([i] * len(x['id']))
  xs.append(x)
x = gezi.merge_array_dicts(xs)
oof = pd.DataFrame(x)
oof

,id,label,anchor,target,text,context,fold,pred
0,37d61fd2272659b1,0.50,abatement,abatement of pollution,abatement[SEP]abatement of pollution[SEP]HUMAN...,A47,0,1.387905
1,7b9652b17b68b7a4,0.75,abatement,act of abating,abatement[SEP]act of abating[SEP]HUMAN NECESSI...,A47,0,2.914510
2,36d72442aefd8232,0.25,abatement,active catalyst,abatement[SEP]active catalyst[SEP]HUMAN NECESS...,A47,0,-2.715760
3,5296b0c19e1ce60e,0.50,abatement,eliminating process,abatement[SEP]eliminating process[SEP]HUMAN NE...,A47,0,1.178741
4,54c1e3b9184cb5b6,0.00,abatement,forest region,abatement[SEP]forest region[SEP]HUMAN NECESSIT...,A47,0,-5.642568
...,...,...,...,...,...,...,...,...
36468,48012024e32a4d49,0.50,well pump,well drilling pump,well pump[SEP]well drilling pump[SEP]MECHANICA...,F04,4,2.488477
36469,539d91c11caf877d,0.75,well pump,well motor,well pump[SEP]well motor[SEP]MECHANICAL ENGINE...,F04,4,2.685757
36470,322c33cfb1449a12,0.75,well pump,well pump motor,well pump[SEP]well pump motor[SEP]MECHANICAL E...,F04,4,2.893347
36471,bac023caed41dc15,0.50,well pump,well system,well pump[SEP]well system[SEP]MECHANICAL ENGIN...,F04,4,1.718366


In [5]:
df = df.merge(oof[['id', 'pred']], on=['id'])
df

,id,anchor,target,context,score,sector,targets,targets_list,context_text,text,num_words,cat,relevance,group,fold,worker,pred
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A,act of abating; active catalyst; eliminating p...,"[act of abating, active catalyst, eliminating ...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]abatement of pollution[SEP]HUMAN...,64,2,0.666667,abatement,0,40,1.387905
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A,abatement of pollution; active catalyst; elimi...,"[abatement of pollution, active catalyst, elim...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]act of abating[SEP]HUMAN NECESSI...,64,3,0.888889,abatement,0,25,2.914510
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A,abatement of pollution; act of abating; elimin...,"[abatement of pollution, act of abating, elimi...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]active catalyst[SEP]HUMAN NECESS...,64,1,0.333333,abatement,0,70,-2.715760
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A,abatement of pollution; act of abating; active...,"[abatement of pollution, act of abating, activ...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]eliminating process[SEP]HUMAN NE...,64,2,0.666667,abatement,0,20,1.178741
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A,abatement of pollution; act of abating; active...,"[abatement of pollution, act of abating, activ...",HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]forest region[SEP]HUMAN NECESSIT...,64,0,0.000000,abatement,0,5,-5.642568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden article[SEP]PERFORMING...,61,4,1.000000,wood article,0,65,7.813949
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden box[SEP]PERFORMING OPE...,61,2,0.666667,wood article,0,10,-6.325552
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden handle[SEP]PERFORMING ...,61,2,0.666667,wood article,0,35,-0.383295
36471,756ec035e694722b,wood article,wooden material,B44,0.75,B,article generator; article spinner; article sp...,"[article generator, article spinner, article s...",PERFORMING OPERATIONS; TRANSPORTING. DECORATIV...,wood article[SEP]wooden material[SEP]PERFORMIN...,61,3,0.888889,wood article,0,60,2.496070


In [6]:
df = df[['id', 'fold', 'anchor', 'target', 'context', 'sector', 'score', 'pred']]

In [7]:
calc_metric(df.score.values, normalize(df.pred.values))

0.8503665422716756

In [8]:
df0 = df[df.fold==0]

In [9]:
calc_metric(df0.score.values, normalize(df0.pred.values))

0.8668121907009169

In [10]:
calc_metric(df0.score.values, df0.pred.values)

0.8668121909039002

In [11]:
df0['normed_pred'] = normalize(df0.pred.values)

In [12]:
df0

,id,fold,anchor,target,context,sector,score,pred,normed_pred
0,37d61fd2272659b1,0,abatement,abatement of pollution,A47,A,0.50,1.387905,0.553474
1,7b9652b17b68b7a4,0,abatement,act of abating,A47,A,0.75,2.914510,0.653174
2,36d72442aefd8232,0,abatement,active catalyst,A47,A,0.25,-2.715760,0.285470
3,5296b0c19e1ce60e,0,abatement,eliminating process,A47,A,0.50,1.178741,0.539814
4,54c1e3b9184cb5b6,0,abatement,forest region,A47,A,0.00,-5.642568,0.094324
...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,0,wood article,wooden article,B44,B,1.00,7.813949,0.973149
36469,42d9e032d1cd3242,0,wood article,wooden box,B44,B,0.50,-6.325552,0.049720
36470,208654ccb9e14fa3,0,wood article,wooden handle,B44,B,0.50,-0.383295,0.437800
36471,756ec035e694722b,0,wood article,wooden material,B44,B,0.75,2.496070,0.625847


In [13]:
df[(df.anchor=='abatement') & (df.sector=='H')]

,id,fold,anchor,target,context,sector,score,pred
47,09db0a872aa1d88c,0,abatement,noise abatement,H01,H,0.5,0.617576
48,6bdd1d05ffa3401e,0,abatement,emission abatement,H04,H,0.5,0.828582


In [14]:
df[df.anchor=='abatement']

,id,fold,anchor,target,context,sector,score,pred
0,37d61fd2272659b1,0,abatement,abatement of pollution,A47,A,0.50,1.387905
1,7b9652b17b68b7a4,0,abatement,act of abating,A47,A,0.75,2.914510
2,36d72442aefd8232,0,abatement,active catalyst,A47,A,0.25,-2.715760
3,5296b0c19e1ce60e,0,abatement,eliminating process,A47,A,0.50,1.178741
4,54c1e3b9184cb5b6,0,abatement,forest region,A47,A,0.00,-5.642568
5,067203128142739c,0,abatement,greenhouse gases,A47,A,0.25,-2.791697
6,061d17f04be2d1cf,0,abatement,increased rate,A47,A,0.25,-5.481939
7,e1f44e48399a2027,0,abatement,measurement level,A47,A,0.25,-3.913082
8,0a425937a3e86d10,0,abatement,minimising sounds,A47,A,0.50,-4.790408
9,ef2d4c2e6bbb208d,0,abatement,mixing core materials,A47,A,0.25,-3.087733


In [15]:
df.describe(PERCENTILES2)

,fold,score,pred
count,36473.000000,36473.000000,36473.000000
mean,2.040222,0.362062,-0.937649
std,1.434331,0.258335,3.312330
min,0.000000,0.000000,-7.577183
25%,1.000000,0.250000,-2.904185
50%,2.000000,0.250000,-1.000042
75%,3.000000,0.500000,1.176914
90%,4.000000,0.750000,2.883717
95%,4.000000,0.750000,4.497497
99%,4.000000,1.000000,8.367338


In [16]:
df0[(df0.anchor=='abatement') & (df0.sector=='H')]

,id,fold,anchor,target,context,sector,score,pred,normed_pred
47,09db0a872aa1d88c,0,abatement,noise abatement,H01,H,0.5,0.617576,0.503165
48,6bdd1d05ffa3401e,0,abatement,emission abatement,H04,H,0.5,0.828582,0.516945


In [17]:
df0_preds = df0.groupby(['anchor', 'context'])['pred'].apply(list).reset_index(name='preds')

In [18]:
df0_preds

,anchor,context,preds
0,abatement,A47,"[1.3879051208496094, 2.9145100116729736, -2.71..."
1,abatement,A61,"[-1.0380666255950928, -0.37499597668647766, -1..."
2,abatement,A62,[0.4268309772014618]
3,abatement,C01,[0.5275442600250244]
4,abatement,F16,[-0.7307510375976562]
...,...,...,...
322,wearability,A61,"[-1.4835290908813477, -2.3347537517547607, -0...."
323,wearability,B21,"[3.031459331512451, 3.8226864337921143, -1.927..."
324,wood article,B05,"[0.36969539523124695, -6.842830181121826, -6.8..."
325,wood article,B27,[-0.12219017744064331]


In [19]:
df0_preds2 = df0.groupby(['anchor', 'context'])['normed_pred'].apply(list).reset_index(name='preds')
df0_preds2

,anchor,context,preds
0,abatement,A47,"[0.5534740090370178, 0.6531742811203003, 0.285..."
1,abatement,A61,"[0.395037442445755, 0.43834158778190613, 0.367..."
2,abatement,A62,[0.4907076954841614]
3,abatement,C01,[0.4972851276397705]
4,abatement,F16,[0.41510775685310364]
...,...,...,...
322,wearability,A61,"[0.3659449517726898, 0.31035274267196655, 0.39..."
323,wearability,B21,"[0.6608120203018188, 0.7124859094619751, 0.336..."
324,wood article,B05,"[0.4869762659072876, 0.015937119722366333, 0.0..."
325,wood article,B27,[0.4548519551753998]


In [20]:
df0_scores = df0.groupby(['anchor', 'context'])['score'].apply(list).reset_index(name='scores')
df0_scores

,anchor,context,scores
0,abatement,A47,"[0.5, 0.75, 0.25, 0.5, 0.0, 0.25, 0.25, 0.25, ..."
1,abatement,A61,"[0.5, 0.5, 0.25]"
2,abatement,A62,[0.5]
3,abatement,C01,[0.5]
4,abatement,F16,[0.0]
...,...,...,...
322,wearability,A61,"[0.5, 0.25, 0.5, 0.0, 0.25, 0.25, 0.25, 0.25, ..."
323,wearability,B21,"[0.75, 0.75, 0.75, 0.25, 0.25, 0.0, 0.25, 0.75..."
324,wood article,B05,"[0.5, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0...."
325,wood article,B27,[0.5]


In [21]:
df_scores = df.groupby(['anchor', 'context'])['score'].apply(list).reset_index(name='scores')

In [22]:
df_scores

,anchor,context,scores
0,abatement,A47,"[0.5, 0.75, 0.25, 0.5, 0.0, 0.25, 0.25, 0.25, ..."
1,abatement,A61,"[0.5, 0.5, 0.25]"
2,abatement,A62,[0.5]
3,abatement,C01,[0.5]
4,abatement,F16,[0.0]
...,...,...,...
1694,wiring trough,F16,"[0.25, 0.5, 0.5, 0.25, 0.5, 0.25, 0.25, 0.25, ..."
1695,wiring trough,H02,"[0.25, 0.25, 0.5, 0.5, 0.5, 0.25, 0.0, 0.25, 0..."
1696,wood article,B05,"[0.5, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0...."
1697,wood article,B27,[0.5]


In [23]:
df0['normed_pred2'] = np.clip(df0.pred, -5, 5)

In [24]:
calc_metric(df0.score.values, df0.pred.values)

0.8668121909039002

In [28]:
with open('../input/periodictable/periodic_table.p', 'rb') as fin:
    import pickle
    per_table = pickle.load(fin)

def atoms_to_str(atoms):
    return ' '.join([per_table.get(x.lower(), '') for x in atoms])
    
def parse_formula(text):
    import chemparse
    tokenized = text.split(' ')
    
    results = []
    
    for tok in tokenized:
        atoms = chemparse.parse_formula(tok).keys()
        formula = atoms_to_str(atoms)
        if len(formula) < 2 or len(tok) < 3:
            results.append(tok)
        else:
            try:
                f = Formula(tok.upper())
                atoms = f.atoms
                formula = ' '.join([x.name.lower() for x in atoms])
            except Exception as e:
                pass
            
            results.append(formula)
    
    return ' '.join(results)
 
def parse_df_formulas(df):
    df = df.copy()
    df.loc[:, 'target'] = df.target.apply(parse_formula)
    return df

In [29]:
train_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
train_df2 = parse_df_formulas(train_df)
train_df2[train_df2.target != train_df.target].merge(train_df[['id', 'target']], on='id', suffixes=('_replaced', '_original'))

,id,anchor,target_replaced,context,score,target_original
0,6c895aabbc9d825a,acid absorption,hydrogen absorption,B01,0.50,h2so4 absorption
1,fd3e3273b089947b,alumino silicates,oxygen,C02,0.50,knaalsi3o8
2,7c9aa3081d39cd89,ammonia recovery,nihonium recovery,C01,0.75,nh3 recovery
3,e46453a06c27afea,ammonia recovery,obtain nihonium,C01,0.75,obtain nh3
4,6ce47c9c885de384,ammonia recovery,nihonium recovery,C10,0.75,nh3 recovery
5,b47610a82b8a74b5,conh2,hydrogen nitrogen co moiety,C09,0.75,h2n co moiety
6,c700bf31b6e48d2b,consist of oxalic acid,consist of carbon hydrogen oxygen,C04,0.75,consist of c2h2o4
7,621d1f99090b0938,consist of oxalic acid,holmium hydrogen,C23,0.50,ho2cco2h
8,d943fe79855645b7,fluorinated olefin,carbon hydrogen fluorine,C08,0.50,c6h3f9
9,a4a32a766bfdc782,her2 targeted,cadmium,A61,0.50,cd340


In [31]:
df[df.id=='6c895aabbc9d825a']

,id,fold,anchor,target,context,sector,score,pred
285,6c895aabbc9d825a,1,acid absorption,h2so4 absorption,B01,B,0.5,2.816246


In [32]:
train_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
train_df2 = parse_df_formulas(train_df)

In [34]:
train_df2[train_df2.id=='6c895aabbc9d825a']

,id,anchor,target,context,score
285,6c895aabbc9d825a,acid absorption,hydrogen absorption,B01,0.5
